In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader

The Centre for the Studies of Apshyxia and Resuscitation
---

In [11]:
model_local = ChatOllama(model="mistral",temperature=0.0)

In [12]:
loader = BSHTMLLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\llm_testing_urls\\test_1.html")

data = loader.load()
print(data)

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   Addresses and phone numbers - About Us - Mayo Clinic\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n     This content does not have an English version.\n    \n\n\n\n\n\n     This content does not have an Arabic version.\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n         \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                          Care at \n Mayo Clinic\n                         \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                  Patient-Centered Care\n                                 \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                  About Mayo Clinic\n                                 \n\n\n\n\n\n\

In [13]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(data)

In [14]:
doc_splits

[Document(page_content='Addresses and phone numbers - About Us - Mayo Clinic\n  \n\n     This content does not have an English version.\n    \n\n     This content does not have an Arabic version.\n    \n\n\n          Skip to content\n         \n\n                          Care at \n Mayo Clinic\n                         \n\n                                  Patient-Centered Care\n                                 \n\n\n                                  About Mayo Clinic\n                                 \n\n\n                                  Request Appointment\n                                 \n\n\n                                  Find a Doctor\n                                 \n\n\n                                  Locations', metadata={'source': 'C:\\Users\\Barani\\Desktop\\local_ollama\\llm_testing_urls\\test_1.html', 'title': '\n   Addresses and phone numbers - About Us - Mayo Clinic\n  '}),
 Document(page_content='Locations\n                                 \n\n\n             

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [16]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
    # embedding = HuggingFaceEmbeddings()
)
retriever = vectorstore.as_retriever()

In [7]:
print("Before RAG\n")
before_rag_template = "Provide me {topic}"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "International Appointment Office fax of mayo clinic in rochester?"}))

Before RAG

 I apologize for any confusion, but Mayo Clinic does not have an International Appointment Office with a public fax number. Patients or their representatives are encouraged to contact the Mayo Clinic Appointment Scheduling line at +1-507-284-2590 or toll-free at 1-800-533-1515 for scheduling appointments and obtaining additional information. Alternatively, you can use the online appointment request form on Mayo Clinic's website: https://www.mayoclinic.org/appointments/request-an-appointment/index.jsp.

For international patients seeking consultation or second opinions, Mayo Clinic offers virtual consultations through video conferencing technology. You can learn more about this service and how to schedule a virtual appointment on the Mayo Clinic website: https://www.mayoclinic.edu/for-international-patients/virtual-consultation/gi-003821.html.


In [17]:
# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("provide the International Appointment Office fax from the given context?"))


########
After RAG

 The International Appointment Office fax number is 507-538-7802.
